
# **TG2**






---
---
# Parte 1 : Procesamiento de los datos
---
---

---
# 1. Cargar librerías
---

In [60]:
import numpy as np
import pandas as pd
import altair as alt
from sklearn.preprocessing import LabelEncoder, normalize
from sklearn.model_selection import train_test_split 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVR
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import classification_report, accuracy_score

alt.data_transformers.enable('default', max_rows=None)

DataTransformerRegistry.enable('default')

---
# 2. Cargar dataset
---

In [29]:
from google.colab import files
import io

uploaded = files.upload()


Saving Pokemon.csv to Pokemon (1).csv


In [48]:
df = pd.read_csv(io.BytesIO(uploaded['Pokemon.csv']))

df.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [31]:
df.columns

Index(['#', 'Name', 'Type 1', 'Type 2', 'Total', 'HP', 'Attack', 'Defense',
       'Sp. Atk', 'Sp. Def', 'Speed', 'Generation', 'Legendary'],
      dtype='object')

In [32]:
df.shape

(800, 13)

In [33]:
df["Generation"].unique()

array([1, 2, 3, 4, 5, 6])

In [34]:
df["Type 1"].unique()

array(['Grass', 'Fire', 'Water', 'Bug', 'Normal', 'Poison', 'Electric',
       'Ground', 'Fairy', 'Fighting', 'Psychic', 'Rock', 'Ghost', 'Ice',
       'Dragon', 'Dark', 'Steel', 'Flying'], dtype=object)

In [35]:
df["Type 2"].unique()

array(['Poison', nan, 'Flying', 'Dragon', 'Ground', 'Fairy', 'Grass',
       'Fighting', 'Psychic', 'Steel', 'Ice', 'Rock', 'Dark', 'Water',
       'Electric', 'Fire', 'Ghost', 'Bug', 'Normal'], dtype=object)

In [36]:
df["Legendary"].unique()

array([False,  True])

### 2.1 Tipos de variables: categorica - numerica

#### Variable categorica

*  Mapping : ideal para variables ordinales donde el orden de las categorias es importante.
* Label Encoding
* OneHot Encoding
* Binary Encoding



In [39]:
df_A = df.copy(deep=True)

leg_ord_map = {True: 0, False: 1}
df_A["Legendary"] = df_A["Legendary"].map(leg_ord_map)
#df["Legendary"] = df["Legendary"].iloc[0]
df_A.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,1
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,1
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,1
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,1
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,1


In [40]:
df_B = df.copy(deep=True)

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
leg_le = LabelEncoder()
leg_labels = leg_le.fit_transform(df_B['Legendary'])
df_B['Leg_Label'] = leg_labels
df_B.head()


,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Leg_Label
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,0
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False,0
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False,0
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False,0
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False,0


### 2.2 Normalización

In [49]:
df[['Total', 'HP', 'Attack', 'Defense',
       'Sp. Atk', 'Sp. Def', 'Speed']].describe()

,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
count,800.00000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000
mean,435.10250,69.258750,79.001250,73.842500,72.820000,71.902500,68.277500
std,119.96304,25.534669,32.457366,31.183501,32.722294,27.828916,29.060474
min,180.00000,1.000000,5.000000,5.000000,10.000000,20.000000,5.000000
25%,330.00000,50.000000,55.000000,50.000000,49.750000,50.000000,45.000000
50%,450.00000,65.000000,75.000000,70.000000,65.000000,70.000000,65.000000
75%,515.00000,80.000000,100.000000,90.000000,95.000000,90.000000,90.000000
max,780.00000,255.000000,190.000000,230.000000,194.000000,230.000000,180.000000


In [50]:
from sklearn.preprocessing import Normalizer

transformer = Normalizer().fit(df[['Total', 'HP', 'Attack', 'Defense',
       'Sp. Atk', 'Sp. Def', 'Speed']])  
df[['Total', 'HP', 'Attack', 'Defense','Sp. Atk', 'Sp. Def', 'Speed']] = transformer.transform(df[['Total', 'HP', 'Attack', 'Defense',
       'Sp. Atk', 'Sp. Def', 'Speed']])
df = df

df.head()


,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,0.924067,0.130764,0.142388,0.142388,0.188882,0.188882,0.130764,1,False
1,2,Ivysaur,Grass,Poison,0.924672,0.136988,0.141555,0.143838,0.182651,0.182651,0.136988,1,False
2,3,Venusaur,Grass,Poison,0.925136,0.140973,0.144497,0.146260,0.176216,0.176216,0.140973,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,0.923685,0.118232,0.147790,0.181781,0.180303,0.177348,0.118232,1,False
4,4,Charmander,Fire,NaN,0.923809,0.116597,0.155463,0.128556,0.179380,0.149484,0.194329,1,False


In [51]:
df[['Total', 'HP', 'Attack', 'Defense','Sp. Atk', 'Sp. Def', 'Speed']].describe()


,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
count,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000
mean,0.919630,0.148381,0.165749,0.156528,0.151526,0.151648,0.145799
std,0.007234,0.043775,0.046488,0.050395,0.045291,0.039791,0.052796
min,0.839977,0.003801,0.009452,0.009452,0.016633,0.029794,0.008317
25%,0.918610,0.124859,0.134774,0.123238,0.118479,0.123920,0.108078
50%,0.921661,0.142307,0.164497,0.145932,0.149562,0.147191,0.143453
75%,0.923462,0.164463,0.197832,0.179013,0.183599,0.173241,0.182717
max,0.925820,0.472624,0.342074,0.382564,0.327885,0.382564,0.355819


---
---
# Parte 2 : Modelo KNN
---
---

In [0]:
data = df[['Total', 'HP', 'Attack', 'Defense','Sp. Atk', 'Sp. Def', 'Speed']].copy(deep=True)
label = df['Generation'].copy(deep=True)

data_train, data_test, label_train, label_test = train_test_split(data, label, train_size=0.7)


In [0]:
clasificador = KNeighborsClassifier(n_neighbors=15, weights='distance', algorithm='kd_tree')
clasificador.fit(data_train, label_train)
label_predict = clasificador.predict(data_test)


In [65]:
print("Accuracy : ",accuracy_score(label_test, label_predict))

Accuracy :  0.32083333333333336


In [66]:
print(classification_report(label_test, label_predict))

              precision    recall  f1-score   support

           1       0.36      0.41      0.38        49
           2       0.14      0.17      0.15        29
           3       0.42      0.37      0.39        52
           4       0.15      0.12      0.14        33
           5       0.42      0.40      0.41        55
           6       0.29      0.32      0.30        22

    accuracy                           0.32       240
   macro avg       0.30      0.30      0.30       240
weighted avg       0.33      0.32      0.32       240

